In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import json
import os
if '../src' not in sys.path:
    sys.path.append('../src')
import utils
import settings
from retriever import Retriever
from reranker import Reranker
from chatbot import Chatbot
from langchain_openai import ChatOpenAI

In [3]:
logger = utils.get_logger("03_rag")
wrapper = utils.get_text_wrapper()

### Document retriever

The document retriever is responsible for finding relevant documents based on the user question. It initializes an Opensearch client, uses a `neural` query for semantic search.

Use the `MODEL_ID` of the embedding model used at ingestion time to generate vector embeddings for the queries' text.

[OpenSearch Python Client docs](https://opensearch-project.github.io/opensearch-py/api-ref/clients/opensearch_client.html)

In [11]:
retriever = Retriever(logger=logger, index=settings.INDEX_NAME)
reranker = Reranker(logger=logger)

In [5]:
query_text = "Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?"

docs = retriever.retrieve_documents(query_text=query_text)

retriever.print_documents(docs)

{
    "page_content":"| Varianta | Cum dovedesc la înscriere | Este necesar să susțin testul de limbă POLITEHNICA Bucureşti? |\n|:--|:--|:--|\n| Liceu absolvit
în respectiva limbă de predare | Dacă acest lucru nu apare evident din celelalte documente furnizate la înscriere, am grijă să încarc o copie a unui document
eliberat de liceu şi din care să reiasă că am urmat un program de studii în respectiva limbă | NU |\n| Certificat de competențe lingvistice pentru respectiva
limbă de predare (minim B2); | Încarc o copie a certificatului | NU |\n| Disciplină de tip limbă străină promovată în liceu cu media cel puțin 8,00 (pentru
clasele a IX- a,a X-a, a XI-a). | Încarc o copie a unui document eliberat de liceul urmat din care reiese acest lucru | NU |\n| Proba A, B sau C de la examenul
de Bacalaureat; | Încare o copie completă a diplomei de Bacalaureat (inclusiv situația la examen) | NU |\n| Aleg să susțin testul de limbă POLITEHNICA
Bucureşti; | Nu încarc nimic la înscriere | DA |\n| L La 

### RAG pipeline

Initialize llm

In [7]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

llm = ChatOpenAI(model=settings.OPENAI_MODEL, temperature=0, top_p=1)

In [12]:
chatbot = Chatbot(retriever=retriever, reranker=reranker, llm=llm)

In [ ]:
# chatbot.delete_conversation_history()

In [13]:
chatbot.run("Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?")
print(chatbot.refined_question)
# print(chatbot.formatted_prompt)
chatbot._print_response()

Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?
UTILIZATOR:
Ce documente sunt necesare pentru a demonstra cunostintele lingvistice la inscrierea la un program de studii in limba engleza?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru a demonstra cunoștințele lingvistice la înscrierea la un program de studii în limba engleză, candidații pot prezenta următoarele documente:

1. **Diploma
de liceu** în care a fost urmat un program de studii în limba engleză, dacă acest lucru nu este evident din celelalte documente furnizate la înscriere.
2.
**Certificat de competențe lingvistice** pentru limba engleză, cu un nivel minim B2.
3. **Foaia matricolă** care să ateste promovarea unei discipline de tip
„limbă străină” în liceu, cu media cel puțin 8,00, pentru clasele a IX-a, a X-a și a XI-a.

In [14]:
chatbot.run("Cand are loc testul online de limba de la politehnica?")
print(f"Refined question: {chatbot.refined_question}")
# print(chatbot.formatted_prompt)
chatbot._print_response()

Refined question: Când se va desfășura testul online de limbă străină organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cand are loc testul online de limba de la politehnica?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Testul online de limbă străină organizat de Universitatea Politehnica din București se va desfășura în două sesiuni. Prima sesiune va avea loc pe data de 21
martie 2025, iar a doua sesiune pe 23 mai 2025. Este important de menționat că data de 23 mai 2025 este valabilă doar pentru candidații înscriși la facultățile
care aplică admiterea anticipată pe baza rezultatelor la activitățile școlare (dosar). Detalii suplimentare privind organizarea testului pot fi găsite pe
site-ul oficial al universității.

Surse:
- Regulament admitere licență 2025: https://upb.ro/wp-content/uploads/2024/12/Regulament-admitere_licenta_2025.pdf
(pagina 26

In [15]:
chatbot.run("Cum ma inscriu la test?")
print(f"Refined question: {chatbot.refined_question}")
chatbot._print_response()

Refined question: Cum pot să mă înscriu la testul online de limbă străină organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cum ma inscriu la test?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru a te înscrie la testul online de limbă străină organizat de Universitatea Politehnica din București, trebuie să urmezi următorii pași:

1. **Înscriere**:
Completează formularul de înscriere disponibil la adresa https://forms.office.com/e/2AWesNbYr9. Este important ca adresa de e-mail pe care o folosești în
formular să fie aceeași cu cea utilizată în procesul de înscriere la concurs de pe pagina http://admitere.pub.ro/. Formularul trebuie completat până cel târziu
pe 21 mai 2025, ora 12:00. Toate câmpurile din formular sunt obligatorii, iar candidații care nu completează formularul la timp și în integralitate nu vor putea
susține testul.

2. **Obținer

In [ ]:
# chatbot.delete_last_message()

In [16]:
docs = chatbot.state["documents"]
for doc in docs:
    print(json.dumps(doc.metadata, indent=4, ensure_ascii=False))

{
    "score": 0.09848484,
    "filename": "Test-limba-straina-23-05-detalii-S1-2025.pdf",
    "page_number": 1,
    "id": "4a8af42a6c47f11b1d13d802bee4da49f7d449950fa3f74b338baa449e1e595b-3",
    "text": "româna II. Am ales să susțin testul, ce trebuie să fac? 1. Mă înscriu (anunț cine sunt și pentru ce limbă susțin testul). 2. Obțin accesul la platforma on-line de test. 3. Susțin testul. 4. Primesc rezultatul. III.Etapele testului on-line POLITEHNICA București A – Înscriere Candidații trebuie să se înscrie pentru a putea susține testul on-line POLITEHNICA București din data de 23 mai 2025 prin completarea formularului de la adresa https://forms.office.com/e/2AWesNbYr9 Adresa de e-mail completată in formular trebuie să fie aceeași adresa de e-mail pe care candidatul a utilizat-o sau o va utiliza în cadrul procesului de înscriere la concurs de pe pagina http://admitere.pub.ro/. Formularul trebuie completat până cel târziu pe 21 mai 2025 ora 12:00. Toate câmpurile din formular sunt obli